# Optuna Study – Bi‑LSTM Hyper‑Parameters

In [ ]:
import optuna, os, tensorflow as tf, numpy as np
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
def get_data(MAX_VOCAB=25_000, MAX_LEN=120):
    df = load_dataset("liar", split="train").to_pandas()[["statement","label"]]
    df.columns = ["text","target"]
    df.target = df.target.apply(lambda x: 0 if x in [0,1,2] else 1)
    tok = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
    tok.fit_on_texts(df.text)
    X = pad_sequences(tok.texts_to_sequences(df.text), maxlen=MAX_LEN, padding="post")
    y = df.target.values
    return tok, *train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
tok, X_tr, X_te, y_tr, y_te = get_data()

def objective(trial):
    emb_dim    = trial.suggest_categorical("emb_dim",[64,128,256])
    lstm_units = trial.suggest_int("lstm_units",32,128,step=32)
    dropout    = trial.suggest_float("dropout",0.1,0.5,step=0.1)
    lr         = trial.suggest_float("lr",1e-4,5e-3,log=True)

    model = Sequential([
        Embedding(len(tok.word_index)+1, emb_dim, mask_zero=True),
        Bidirectional(LSTM(lstm_units)),
        Dropout(dropout),
        Dense(1, activation="sigmoid")
    ])
    model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(lr), metrics=["accuracy"])
    hist = model.fit(X_tr, y_tr, epochs=3, batch_size=128, validation_split=0.2, verbose=0)
    val_acc = max(hist.history["val_accuracy"])
    tf.keras.backend.clear_session()
    return val_acc

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20, show_progress_bar=True)
study.best_params, study.best_value

In [ ]:
best = study.best_params
emb_dim, lstm_units, dropout, lr = best["emb_dim"],best["lstm_units"],best["dropout"],best["lr"]
tok_full, X_tr, X_te, y_tr, y_te = get_data()
model = Sequential([
    Embedding(len(tok_full.word_index)+1, emb_dim, mask_zero=True),
    Bidirectional(LSTM(lstm_units)),
    Dropout(dropout),
    Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(lr), metrics=["accuracy"])
model.fit(X_tr, y_tr, epochs=5, batch_size=128, validation_split=0.1, verbose=0)
os.makedirs("models", exist_ok=True)
model.save("models/bi_lstm_fake_news.h5")
import pickle
with open("models/tokenizer.pkl", "wb") as f:
    pickle.dump(tok_full, f)
print("🚀 Saved tuned model to models/bi_lstm_fake_news.h5")